# Data Organization
This notebook will read in each data file provided from kaggle and place it into a single CSV file for analysis. The main dataframe is attatched, so no need to re-run this notebook as it takes a very long time to parse all the data together. The main dataframe is housed in "master_chrono.csv".

In [2]:
#Supress Warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

#Set by user
file_path = '/Users/admin/Desktop/data_bowl/data_bowl_github/raw_data/'


In [3]:
#Get all the data into the files
week_tot = np.arange(1,9)


df = pd.concat([pd.read_csv(file_path+'week{}.csv'.format(wk)) for wk in np.arange(1,9)])

#Group players by their ID
players_df = pd.read_csv(file_path+'players.csv')
players_df = players_df.append({'nflId':-999,'displayName':'Football'},ignore_index=True)
players_df.set_index('nflId',inplace=True)


In [5]:
#Make a column of player names -> Iterate through every data point (May take some time)
nflId = df['nflId'].values
nflId[np.isnan(nflId)] = -999

df['players'] = [players_df.loc[val]['displayName'] for val in tqdm(nflId,total=len(nflId))]


100%|████████████████████████████████████████████████████████████████| 8314178/8314178 [10:19<00:00, 13426.44it/s]


In [ ]:
#Add home, away team names and game time
gms = pd.read_csv(file_path+'games.csv')
gms.set_index('gameId',inplace=True)

df['home_name'] = [gms.loc[val]['homeTeamAbbr'] for val in tqdm(df['gameId'].values,total=len(df['gameId'].values))]
df['away_name'] = [gms.loc[val]['visitorTeamAbbr'] for val in tqdm(df['gameId'].values,total=len(df['gameId'].values))]
df['game_time_EST'] = [gms.loc[val]['gameTimeEastern'] for val in tqdm(df['gameId'].values,total=len(df['gameId'].values))]


In [ ]:
#Read in the games.csv file and add this data to the parent dataframe
plys = pd.read_csv(file_path+'games.csv')

cols = plys.columns.tolist()
cols.remove('gameId')
#cols.remove('playId')
cols.remove('season')
cols.remove('gameDate')

#Make empty columns
dct = {}
for c in cols:
    dct[c] = np.zeros((len(df))).astype(object)*np.nan

#Subset the main dataframe
for i in tqdm(range(len(plys))):
    idx = np.where(df['gameId'].values==plys['gameId'].values[i])[0]
    
    #Append the dataframe with this value
    for c in cols:
        dct[c][idx] = plys[c].values[i]
    
for c in cols:
    df[c] = dct[c]


In [ ]:
#Read through the plays.csv file and add this data to the parent dataframe
plys = pd.read_csv(file_path+'plays.csv')

cols = plys.columns.tolist()
cols.remove('gameId')
cols.remove('playId')
cols.remove('playDescription')

#Make empty columns
dct = {}
for c in cols:
    dct[c] = np.zeros((len(df))).astype(object)*np.nan

#Subset the main dataframe
for i in tqdm(range(len(plys))):
    idx = np.where((df['gameId']==plys['gameId'].values[i])&(df['playId']==plys['playId'].values[i]))[0]
    
    #Append the dataframe with this value
    for c in cols:
        dct[c][idx] = plys[c].values[i]
    
for c in cols:
    df[c] = dct[c]


In [ ]:
#Read through the PFF scouting data and add this to the parent dataframe 
plys = pd.read_csv(file_path+'pffScoutingData.csv')

cols = plys.columns.tolist()
cols.remove('gameId')
cols.remove('playId')
cols.remove('nflId')

#Make empty columns
dct = {}
for c in cols:
    dct[c] = np.zeros((len(df))).astype(object)*np.nan

#Subset the main dataframe
for i in tqdm(range(len(plys))):
    idx = np.where((df['gameId']==plys['gameId'].values[i])&(df['playId']==plys['playId'].values[i])&(df['nflId']==plys['nflId'].values[i]))[0]
    
    #Append the dataframe with this value
    for c in cols:
        dct[c][idx] = plys[c].values[i]
    
for c in cols:
    df[c] = dct[c]
    

In [ ]:
#Read in the play-by-play csv file and append this data to the parent dataframe if not already 
#The R script to obtain this publicly available data is attatched in "get_pbp_data.R" (see https://github.com/nflverse/nflfastR)

plys = pd.read_csv(file_path+'pbp.csv')
cols = plys.columns.tolist()[8:]
irm = []
for c in cols:
    if c in df.columns:
        irm.append(np.where(np.array(cols)==c)[0][0])
ikeep = np.setdiff1d(np.arange(len(cols)),irm)
cols = np.array(cols)[ikeep].tolist()

#Make empty columns
dct = {}
for c in tqdm(cols,total=len(cols)):
    dct[c] = np.zeros((len(df))).astype(object)*np.nan
    
#Subset the main dataframe
for i in tqdm(range(len(plys))):
    idx = np.where((df['gameId']==plys['old_game_id'].values[i])&(df['playId']==plys['play_id'].values[i]))[0]
    
    #Append the dataframe with this value
    for c in cols:
        dct[c][idx] = plys[c].values[i]
    
for c in cols:
    df[c] = dct[c]
    

In [ ]:
#Add official position to the main dataframe
plys = pd.read_csv(file_path+'players.csv')
cols = ['officialPosition']

#Make empty columns
dct = {}
for c in cols:
    dct[c] = np.zeros((len(df))).astype(object)*np.nan

#Subset the main dataframe
for i in tqdm(range(len(plys))):
    idx = np.where(df['nflId']==plys['nflId'].values[i])[0]
    
    #Append the dataframe with this value
    for c in cols:
        dct[c][idx] = plys[c].values[i]
    
for c in cols:
    df[c] = dct[c]

In [ ]:
#Organize the dataset chronologically

#Make a unique ID for each
unique_id = ['{}_{}_{}'.format(df['gameId'].values[i],datetime.strptime(df['time'].values[i],'%Y-%m-%d %H:%M:%S').strftime('%Y%m%d_%H%M%S'),df['frameId'].values[i]) for i in tqdm(range(len(df)))]
arg_id = np.argsort(np.array(unique_id))
d_sort = df.loc[arg_id]

#Write the main dataframe to the home directory
df.to_csv('master_chrono.csv',mode='w')
